In [2]:
import libdatasets
from libactive import active_split
from libdatasets import *
import librun
import libstop
from sklearn.utils import check_random_state
from libadversarial import uncertainty_stop
from libstop import rank_stop_conds, reconstruct_unlabelled
from dotenv import load_dotenv

load_dotenv()

True

In [64]:
matrix = {
    # Dataset fetchers should cache if possible
    # Lambda wrapper required for function to be pickleable (sent to other threads via joblib)
    "datasets": [
        # These passed (without sparse equality check)
        #("rcv1", wrap(rcv1, None)),
        #("webkb", wrap(webkb, None)),
        #("spamassassin", wrap(spamassassin, None)),
        
        #("avila", wrap(avila, None)),
        #("smartphone", wrap(smartphone, None)),
        ("swarm", wrap(swarm, None)),
        ("sensorless", wrap(sensorless, None)),
        ("splice", wrap(splice, None)),
        ("anuran", wrap(anuran, None)),
    ],
    "dataset_mutators": {
        "none": (lambda *x, **kwargs: x),
    },
    "methods": [
        ("uncertainty", partial(uncertainty_stop, n_instances=10)),
    ],
    "models": [
        "svm-linear",
        #"random-forest",
    ],
    "meta": {
        "dataset_size": 1000,
        "labelled_size": 10,
        "test_size": 0.5,
        "n_runs": 10,
        "ret_classifiers": True,
        "ensure_y": True,
        "stop_info": True,
        "aggregate": False,
        "stop_function": (
            "len1000",
            lambda learner: learner.y_training.shape[0] >= 1000,
        ),
        "pool_subsample": 1000,
    },
}

_results = librun.run(matrix, force_cache=True, fragment_run_start=0, fragment_run_end=9)
results_plots = [result[0] for result in _results]
classifiers = [result[1] for result in _results]
classifiers = [clf for clf in classifiers]

['/home/zac/.cache/pypoetry/virtualenvs/research-vDwbEF2m-py3.8/lib/python3.8/site-packages/ipykernel_launcher.py', '-f', '/home/zac/.local/share/jupyter/runtime/kernel-9df04daa-8eec-4a21-a6f2-c6c011d89bb8.json']


Experiment:   0%|          | 0/4 [00:00<?, ?it/s]

In [67]:
from importlib import reload; reload(libstop); from libstop import reconstruct_unlabelled
for (conf, results), clfs in zip(results_plots, classifiers):
    print(conf.dataset_name)
    X, y = getattr(libdatasets, conf.dataset_name)(None)
    print("  is sparse" if isinstance(X, scipy.sparse.csr_matrix) else "  is dense")
    
    for i in conf.runs:
        print(f"  {i}")
        _, X_unlabelled, _, Y_oracle, _, _ = active_split(
            X, y, labeled_size=conf.meta['labelled_size'], test_size=conf.meta['test_size'], random_state=check_random_state(i), ensure_y=conf.meta['ensure_y']
        )
        try:
            size = None
            for pool in reconstruct_unlabelled(clfs[i], X_unlabelled, Y_oracle, dense_atol=1e-1 if conf.dataset_name =='swarm' else 1e-6):
                if size is None:
                    size = pool.shape[0]
                    print(f"   Initial size {size}")
                else:
                    assert pool.shape[0] == size-10, f"{pool.shape[0]} == {size-10}"
                    size = pool.shape[0]
        except Exception as e:
            print(f'   FAILED: {e}')
            raise e

swarm
  is dense
  0
   Initial size 11997
  1
   Initial size 11997
  2
   Initial size 11997
  3
   Initial size 11997
  4
   Initial size 11997
  5
   Initial size 11997
  6
   Initial size 11997
  7
   Initial size 11997
  8
   Initial size 11997
  9
   Initial size 11997
sensorless
  is dense
  0
   Initial size 29243
  1
   Initial size 29243
  2
   Initial size 29243
  3
   Initial size 29243
  4
   Initial size 29243
  5
   Initial size 29243
  6
   Initial size 29243
  7
   Initial size 29243
  8
   Initial size 29243
  9
   Initial size 29243
splice
  is sparse
  0
   Initial size 1585
  1
   Initial size 1585
  2
   Initial size 1585
  3
   Initial size 1585
  4
   Initial size 1585
  5
   Initial size 1585
  6
   Initial size 1585
  7
   Initial size 1585
  8
   Initial size 1585
  9
   Initial size 1585
anuran
  is dense
  0
   Initial size 3587
  1
   Initial size 3587
  2
   Initial size 3587
  3
   Initial size 3587
  4
   Initial size 3587
  5
   Initial size 3587
  6


In [ ]:
clf.y_training

In [46]:
clf.X_training

array([[ 2.6607400e-01,  5.8024200e-01,  1.1470900e-01, -1.6546900e-01,
         2.6171800e-01, -1.8340900e-01,  1.8242600e-01,  4.5164100e-01,
         4.0708000e-01, -1.5086900e-01],
       [-1.2892900e-01,  1.2655362e+01, -1.6269000e-01,  1.9930150e+00,
        -3.2240300e+00, -5.8481600e-01, -6.1033300e-01,  7.6343200e-01,
         1.1658290e+00, -1.5845200e-01],
       [ 1.3029200e-01, -1.5776900e-01,  5.7704100e-01, -6.7098600e-01,
         1.7234000e-01, -1.2630550e+00,  3.1425000e-02,  2.1826700e-01,
         4.0708000e-01, -1.0523880e+00],
       [-1.9064800e-01, -1.0920600e+00,  4.0277800e-01,  7.4942500e-01,
         1.7234000e-01,  8.5471200e-01, -2.3282800e-01,  1.9261900e-01,
         7.5007600e-01,  1.0322940e+00],
       [-5.8565100e-01, -3.5404900e-01, -3.2105280e+00,  3.5651700e-01,
         2.6171800e-01,  2.7336400e-01,  5.9768100e-01,  1.4125090e+00,
         1.4048870e+00,  5.7323000e-02],
       [ 5.6229000e-02, -9.4960000e-02,  2.2851400e-01, -6.5806600e-01,
   

In [51]:
clf = clfs[i][1]
clf.X_training.shape

(22, 10)

In [52]:
X_unlabelled.shape

(10421, 10)

In [53]:
X_unlabelled[
    (X_unlabelled[:, np.newaxis] != clf.X_training).all(-1).any(-1)
].copy().shape

(10421, 10)

In [56]:
np.count_nonzero(X_unlabelled == clf.X_training[0])

6239